In [150]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [151]:
data=pd.read_csv("Churn_Modelling.csv")


In [152]:
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [153]:
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [154]:
#Encoding categorical labels
label_encoder_gender=LabelEncoder()
data["Gender"]=label_encoder_gender.fit_transform(data["Gender"])


In [155]:
#one hot encoding for geography
from sklearn.preprocessing import OneHotEncoder
onehot_encoder_geo=OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[["Geography"]]).toarray()
onehot_encoder_geo.get_feature_names_out(['Geography'])


array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [156]:
geo_encoder_df=pd.DataFrame(geo_encoder,columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
data.head()


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [157]:
#concatenating the dataframes
data=pd.concat([data.drop('Geography',axis=1),geo_encoder_df],axis=1)


In [158]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [159]:
#saving the encoder
with open("label_encoder_gender.pkl","wb") as f:
    pickle.dump(label_encoder_gender,f)
with open("onehot_encoder_geo.pkl","wb") as f:
    pickle.dump(onehot_encoder_geo,f)

In [160]:
X=data.drop('Exited',axis=1)
y=data['Exited']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [161]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [162]:
#ANN implementation
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [163]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 Connected wwith input layer
    Dense(32,activation='relu'), ## HL2
    Dense(1,activation='sigmoid')  ## output layer
])

d:\PyProject\ML\pyenv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [164]:
opt=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()
model.compile(optimizer=opt,loss=loss,metrics=['accuracy'])


In [165]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir="logs/fit/" + datetime.datetime.now().strftime("%Y %m %d-%H%M")
tensorflow_callback=TensorBoard(log_dir=log_dir,histogram_freq=1)

In [166]:
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=300,restore_best_weights=True)


In [167]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=400,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8106 - loss: 0.4364 - val_accuracy: 0.8490 - val_loss: 0.3566
Epoch 2/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8503 - loss: 0.3657 - val_accuracy: 0.8585 - val_loss: 0.3558
Epoch 3/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8604 - loss: 0.3453 - val_accuracy: 0.8675 - val_loss: 0.3394
Epoch 4/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8593 - loss: 0.3403 - val_accuracy: 0.8590 - val_loss: 0.3400
Epoch 5/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8659 - loss: 0.3323 - val_accuracy: 0.8590 - val_loss: 0.3492
Epoch 6/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8648 - loss: 0.3351 - val_accuracy: 0.8655 - val_loss: 0.3413
Epoch 7/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8643 - loss: 0.3326 - val_accuracy: 0.8590 - val_loss: 0.3530
Epoch 8/400
250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8580 - loss: 0.3283 - val_accu

In [168]:
model.save('model.h5')

In [1]:
%load_ext tensorboard

In [2]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 2376), started 0:25:45 ago. (Use '!kill 2376' to kill it.)